In [1]:
from master import MASTERModel
import pickle
import numpy as np
import time
import pandas as pd
import yaml
import torch
from qlib.data.dataset import TSDataSampler
from bisect import bisect_left
    
universe = 'csi300'
prefix = 'opensource'
data_dir = f'data/self_exp'
train_data_path = f'{data_dir}/{prefix}/{universe}_self_dl_train.pkl'
valid_data_path = f'{data_dir}/{prefix}/{universe}_self_dl_valid.pkl'
test_data_path  = f'{data_dir}/{prefix}/{universe}_self_dl_test.pkl'

In [2]:
with open(train_data_path, 'rb') as f:
    dl_train = pickle.load(f)
with open(valid_data_path, 'rb') as f:
    dl_valid = pickle.load(f)
with open(test_data_path, 'rb') as f:
    dl_test  = pickle.load(f)

In [3]:
df_train, df_valid, df_test = dl_train.data, dl_valid.data, dl_test.data
assert set(df_train.columns) == set(df_valid.columns) == set(df_test.columns)

In [4]:
assert dl_train.step_len == dl_valid.step_len == dl_test.step_len
step_len = dl_train.step_len

In [5]:
# 验证集与训练集有8天重叠需移除（测试集与验证集同理）
df_valid_no_back = df_valid.loc[df_valid.index.get_level_values(0) > max(df_train.index)[0]]
df_test_no_back  =  df_test.loc[ df_test.index.get_level_values(0) > max(df_valid.index)[0]]

df_all = pd.concat([df_train, df_valid_no_back, df_test_no_back], axis=0)

In [6]:
df_all

feature                     \
                      ($close-$open)/$open ($high-$low)/$open   
datetime   instrument                                           
2008-01-02 SH600000               0.571329           1.780677   
           SH600001               1.762864           2.868712   
           SH600004               3.000000           1.696344   
           SH600005              -1.363176           0.851225   
           SH600006               0.697707           0.583797   
...                                    ...                ...   
2025-02-28 SZ300832               0.474697          -0.051177   
           SZ300896              -1.890484           0.824871   
           SZ300979              -0.522114           0.470806   
           SZ300999              -1.245142           0.110317   
           SZ301269              -2.855090           2.188925   

                                                         \
                      ($close-$open)/($high-$low+1e-12)   
datetime   instrument                                     
2008-01-02 SH600000                            0.254437   
           SH600001                            0.599549   
           SH600004                            1.438916   
           SH600005                           -0.825221   
           SH600006                            0.471072   
...                                                 ...   
2025-02-28 SZ300832                            0.441336   
           SZ300896                           -1.156216   
           SZ300979                           -0.370571   
           SZ300999                           -1.056346   
           SZ301269                           -1.139217   

                                                            \
                      ($high-Greater($open, $close))/$open   
datetime   instrument                                        
2008-01-02 SH600000                               3.000000   
           SH600001                               2.966559   
           SH600004                              -0.640195   
           SH600005                              -0.250936   
           SH600006                               0.188684   
...                                                    ...   
2025-02-28 SZ300832                               1.172513   
           SZ300896                               0.496950   
           SZ300979                               2.792662   
           SZ300999                              -0.096285   
           SZ301269                               0.044907   

                                                                         \
                      ($high-Greater($open, $close))/($high-$low+1e-12)   
datetime   instrument                                                     
2008-01-02 SH600000                                            0.949831   
           SH600001                                            0.399878   
           SH600004                                           -0.918490   
           SH600005                                           -0.598857   
           SH600006                                           -0.211728   
...                                                                 ...   
2025-02-28 SZ300832                                            1.137968   
           SZ300896                                           -0.085367   
           SZ300979                                            1.887664   
           SZ300999                                           -0.253801   
           SZ301269                                           -0.636839   

                                                        \
                      (Less($open, $close)-$low)/$open   
datetime   instrument                                    
2008-01-02 SH600000                           2.706018   
           SH600001                           2.451650   
           SH600004                          -1.059098   
           SH600005          

In [7]:
trade_dates = sorted(set(df_all.index.get_level_values(0)))

def slice(df, start, end, back):
    real_start_idx = bisect_left(trade_dates, pd.Timestamp(start)) - back
    real_start = trade_dates[real_start_idx]
    
    cond1 = df.index.get_level_values(0) <= end
    cond2 = df.index.get_level_values(0) >= real_start
    return df.loc[cond1 & cond2]

In [8]:
def build_new_dataset(train_date_range, valid_date_range, test_date_range):
    df_train_new = slice(df_all, *train_date_range, back=0)
    df_valid_new = slice(df_all, *valid_date_range, back=8)
    df_test_new  = slice(df_all, *test_date_range, back=8)

    dl_train_new = TSDataSampler(data=df_train_new,
                                 start=train_date_range[0],
                                 end=train_date_range[1],
                                 step_len=dl_train.step_len,
                                 fillna_type=dl_train.fillna_type)
    dl_valid_new = TSDataSampler(data=df_valid_new,
                                 start=valid_date_range[0],
                                 end=valid_date_range[1],
                                 step_len=dl_valid.step_len,
                                 fillna_type=dl_valid.fillna_type)
    dl_test_new  = TSDataSampler(data=df_test_new,
                                 start=test_date_range[0],
                                 end=test_date_range[1],
                                 step_len=dl_test.step_len,
                                 fillna_type=dl_test.fillna_type)
    return dl_train_new, dl_valid_new, dl_test_new

In [9]:
# extend 1 year
train_date_range = ('2008-01-01', '2021-09-30')
valid_date_range = ('2021-10-01', '2021-12-31')
test_date_range  = ('2022-01-01', '2025-02-28')

dl_train_new, dl_valid_new, dl_test_new = build_new_dataset(train_date_range,
                                                            valid_date_range,
                                                            test_date_range)

new_prefix = 'extend_1'
train_data_path_new = f'{data_dir}/{new_prefix}/{universe}_extend_dl_train.pkl'
valid_data_path_new = f'{data_dir}/{new_prefix}/{universe}_extend_dl_valid.pkl'
test_data_path_new  = f'{data_dir}/{new_prefix}/{universe}_extend_dl_test.pkl'

with open(train_data_path_new, 'wb') as f:
    pickle.dump(dl_train_new, f)
with open(valid_data_path_new, 'wb') as f:
    pickle.dump(dl_valid_new, f)
with open(test_data_path_new, 'wb') as f:
    pickle.dump(dl_test_new, f)

In [10]:
# extend 2 years
train_date_range = ('2008-01-01', '2020-09-30')
valid_date_range = ('2020-10-01', '2020-12-31')
test_date_range  = ('2021-01-01', '2025-02-28')

dl_train_new, dl_valid_new, dl_test_new = build_new_dataset(train_date_range,
                                                            valid_date_range,
                                                            test_date_range)

new_prefix = 'extend_2'
train_data_path_new = f'{data_dir}/{new_prefix}/{universe}_extend_dl_train.pkl'
valid_data_path_new = f'{data_dir}/{new_prefix}/{universe}_extend_dl_valid.pkl'
test_data_path_new  = f'{data_dir}/{new_prefix}/{universe}_extend_dl_test.pkl'

with open(train_data_path_new, 'wb') as f:
    pickle.dump(dl_train_new, f)
with open(valid_data_path_new, 'wb') as f:
    pickle.dump(dl_valid_new, f)
with open(test_data_path_new, 'wb') as f:
    pickle.dump(dl_test_new, f)

In [11]:
# extend 3 years
train_date_range = ('2008-01-01', '2019-09-30')
valid_date_range = ('2019-10-01', '2019-12-31')
test_date_range  = ('2020-01-01', '2025-02-28')

dl_train_new, dl_valid_new, dl_test_new = build_new_dataset(train_date_range,
                                                            valid_date_range,
                                                            test_date_range)

new_prefix = 'extend_3'
train_data_path_new = f'{data_dir}/{new_prefix}/{universe}_extend_dl_train.pkl'
valid_data_path_new = f'{data_dir}/{new_prefix}/{universe}_extend_dl_valid.pkl'
test_data_path_new  = f'{data_dir}/{new_prefix}/{universe}_extend_dl_test.pkl'

with open(train_data_path_new, 'wb') as f:
    pickle.dump(dl_train_new, f)
with open(valid_data_path_new, 'wb') as f:
    pickle.dump(dl_valid_new, f)
with open(test_data_path_new, 'wb') as f:
    pickle.dump(dl_test_new, f)

In [12]:
# extend 5 years
train_date_range = ('2008-01-01', '2017-09-30')
valid_date_range = ('2017-10-01', '2017-12-31')
test_date_range  = ('2018-01-01', '2025-02-28')

dl_train_new, dl_valid_new, dl_test_new = build_new_dataset(train_date_range,
                                                            valid_date_range,
                                                            test_date_range)

new_prefix = 'extend_5'
train_data_path_new = f'{data_dir}/{new_prefix}/{universe}_extend_dl_train.pkl'
valid_data_path_new = f'{data_dir}/{new_prefix}/{universe}_extend_dl_valid.pkl'
test_data_path_new  = f'{data_dir}/{new_prefix}/{universe}_extend_dl_test.pkl'

with open(train_data_path_new, 'wb') as f:
    pickle.dump(dl_train_new, f)
with open(valid_data_path_new, 'wb') as f:
    pickle.dump(dl_valid_new, f)
with open(test_data_path_new, 'wb') as f:
    pickle.dump(dl_test_new, f)